In [1]:
import algorithm as al
import numpy as np
import cv2 as cv
import Queue
from patchdb import *
from psi import *
from copyutils import *
from debug import *
import inspect

[INFO              ] [Logger      ] Record log in C:\Users\xiaoy\.kivy\logs\kivy_16-04-08_28.txt
[INFO              ] [Kivy        ] v1.9.1
[INFO              ] [Python      ] v2.7.11 |Continuum Analytics, Inc.| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
[INFO              ] [Image       ] Providers: img_tex, img_dds, img_gif, img_sdl2 (img_pil, img_ffpyplayer ignored)


In [2]:
def imshow(img):
    cv.imshow('image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
_p = al.Inpainting()

In [4]:
_p.readImage('../../test_images/Kanizsa-triangle-tiny.png', 'source')

_p.readImage('../../test_images/Kanizsa-triangle-mask-tiny.png', 'alpha')

(True, 'Success')

In [5]:
_p._images['filled'] = np.uint8(_p._images['alpha'] > 0)*255
_p._images['inpainted'] = _p._images['source'].copy()

In [6]:
_p._images['source'].shape

(341L, 320L, 3L)

In [7]:
for i in range(0,3):
    _p._images['inpainted'][:,:,i] *= (_p._images['filled']>0)

#
# Step 1a,b: Identify the fill front deltaOmega and compute
#            initial patch priorities

In [8]:
_p.computeBoundaries()
_p.confidenceInitialize()

In [9]:
_p._w = 5

In [10]:
_p._patchDB = PatchDB(_p._images['inpainted'], _p._w, filled=_p._images['filled'])

In [11]:
_p.iterationsInit()
done = False

In [12]:
boundary = _p._boundaryIterator.next()
_p.fillFrontInitialize(boundary, imviewer=None)

In [13]:
boundaryPixels = boundary
# initialize the pixels on the fill front
_p._images['fillFront'] = \
    np.zeros_like(_p._images['filled'], dtype=np.uint8)
_p._images['fillFront'] = \
    cv.drawContours(_p._images['fillFront'],boundaryPixels,-1,255)

# initialize the priority queue with all points on the fill front
_p._deltaOmega = Queue.PriorityQueue()

In [75]:
# col, row = boundaryPixels[10][0]
col, row = (106, 189)
p = PSI((row, col), _p._w, 
          image=_p._images['inpainted'], 
          filled=_p._images['filled'],
          confidence=_p._images['confidence'],
          fillFront=_p._images['fillFront'])
inpaintedImage=_p._images['inpainted'] 
filledImage=_p._images['filled']
confidence=_p._images['confidence']
fillFront=_p._images['fillFront']

In [76]:
inpainted, _ = copyutils.getWindow(inpaintedImage, p._coords, 5)
filled, _ = copyutils.getWindow(filledImage, p._coords, 5)
if len(inpainted.shape) == 3:
    inpainted = cv.cvtColor(inpainted, cv.COLOR_BGR2GRAY)
# Sobel filter
size = 3
Dx = cv.Sobel(src=inpainted, ddepth=cv.CV_32F, dx=0, dy=1, ksize=size)
Dy = -cv.Sobel(src=inpainted, ddepth=cv.CV_32F, dx=1, dy=0, ksize=size)
# Dx *= filled>0
# Dy *= filled>0
d = np.sqrt(Dx**2 + Dy**2)
dmax = d == d.max()
# Dx = Dx[dmax][0]
# Dy = Dy[dmax][0]

In [79]:
inpainted

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0]], dtype=uint8)

In [80]:
filled

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0]], dtype=uint8)

In [81]:
erode_kernel = np.ones((3,3),np.uint8);kernel

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=uint8)

In [82]:
filled_eroded = cv.erode(inpainted, erode_kernel, borderType=cv.BORDER_REPLICATE,iterations=1);filled_eroded

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0]], dtype=uint8)

In [85]:
dx = cv.Sobel(src=inpainted, ddepth=cv.CV_32F, dx=1, dy=0, ksize=3, borderType=cv.BORDER_REPLICATE);dx

array([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [ -255.,  -255.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [ -510.,  -765.,  -255.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [ -255., -1020.,  -765.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,  -765., -1020.,  -255.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,  -255., -1020.,  -765.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,     0.,  -765., -1020.,  -255.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,     0.,  -255., -1020.,  -765.,     0.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,     0.,     0.,  -765., -1020.,  -255.,     0.,     0.,
            0.,     0.,     0.],
       [    0.,     0.,     0.,  -255., -1020.,  -765.,

In [87]:
dx *= filled_eroded > 0

In [88]:
dx

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [37]:
Dx * (filled>0)

array([[        0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.],
       [  3754606.,   1644790.,  -3080783.,  -7054268.,  -8058713.,
         -6530509.,  -4395168.,  -3008806.,  -2480366.,  -2368071.,
         -2358274.],
       [  5120702.,   2068980.,  -5082325., -11896860., -14971477.,
        -14232110., -11990866., -10284819.,  -9575954.,  -9417623.,
         -9404322.],
       [  1441130.,   -910673.,  -7026632., -14351684., -19967748.,
        -22508432., -22759810., -22291298., -21987000., -21916536.,
        -21915998.],
       [ -5648004.,  -6359576.,  -9133050., -14552338., -21382308.,
        -27259872., -30792496., -32277860., -32722296., -32833426.,
        -32857840.],
       [       -0.,        -0.,        -0.,        -0.,        -0.,
               -0.,        -0.,        -0.,        -0.,        -0.,
               -0.],
       [       -0.,        -0.,        -0.,        -0.,   

In [38]:
d

array([[        0.  ,  12613268.  ,  24318762.  ,  29631326.  ,
         24939268.  ,  14594492.  ,   5889934.  ,   1697388.  ,
           477950.  ,    187002.  ,         0.  ],
       [  3754606.  ,  10631455.  ,  21920576.  ,  29140332.  ,
         26545358.  ,  17068156.  ,   8139094.  ,   3682586.5 ,
          2547378.25,   2376234.5 ,   2358274.  ],
       [  5120702.  ,   5728588.5 ,  15825932.  ,  26967014.  ,
         29385972.  ,  22980776.  ,  14964519.  ,  10745647.  ,
          9611682.  ,   9420190.  ,   9404322.  ],
       [  1441130.  ,    912754.75,  10006192.  ,  22962556.  ,
         30361308.  ,  29235404.  ,  24994084.  ,  22631844.  ,
         22011722.  ,  21917782.  ,  21915998.  ],
       [  5648004.  ,   7085883.5 ,   9224807.  ,  18246094.  ,
         27531854.  ,  31490944.  ,  32205790.  ,  32499366.  ,
         32738382.  ,  32834096.  ,  32857840.  ],
       [        0.  ,         0.  ,         0.  ,         0.  ,
                0.  ,         0.  ,      

In [44]:
Dx[dmax][0]

-513.0

In [45]:
Dy[dmax][0]

-959.0